## 4. Transformação de Features de Baseline - Camada Gold


O dataset contém:

* `TRANSACTION_ID`: ID único da transação.
* `TX_DATETIME`: Data e hora da transação.
* `CUSTOMER_ID`: ID do cliente que fez a transação.
* `TERMINAL_ID`: ID do terminal (maquininha de cartão) onde a transação ocorreu.
* `TX_AMOUNT`: Valor da transação.
* `TX_FRAUD`: Label da transação → 1 se for fraude, 0 se for legítima.
* `TX_FRAUD_SCENARIO`:  Label para os tipos de fraudes 


Ou seja, a base é composta por transações simuladas, cada uma com informações mínimas para análise e detecção de fraudes.


Cenário de Fraude (coluna `TX_FRAUD_SCENARIO`)
* 1	Fraudes com valores acima de R$220 — detecção simples baseada em threshold.
* 2	Terminais comprometidos por 28 dias — exige análise temporal por terminal.
* 3	Cartões clonados por 14 dias com transações infladas — exige análise de padrão de gasto por cliente.

### imports


In [0]:
from pyspark.sql.functions import col, to_timestamp, date_format, hour, when, dayofweek

In [0]:
transactions = spark.sql("""
                          SELECT * FROM workspace.fraud_detection.transaction_silver
                          """)


In [0]:
transactions.show()

In [0]:
transactions.printSchema()

### 4.1. transformação de features a partir da coluna `TX_DATETIME`


1. `TX_DURING_WEEKEND`: 1 se a transação ocorreu no fim de semana (sábado ou domingo), 0 caso contrário.

2. `TX_DURING_NIGHT`: 1 se a transação ocorreu entre 00:00h e 06:00h, 0 caso contrário.

In [0]:
# Cria coluna indicando se é fim de semana (1 = sábado ou domingo, 0 = dia útil)
transactions = transactions.withColumn(
    "TX_DURING_WEEKEND",
    when(date_format(("TX_DATETIME"), "E").isin("Sat", "Sun"), 1).otherwise(0)
)

transactions  = transactions.withColumn(
     "TX_DURING_NIGHT",
     when(hour(col("TX_DATETIME")).between(0, 5), 1).otherwise(0)
)

In [0]:
display(transactions.select("TX_DATETIME", "TX_DURING_WEEKEND", "TX_DURING_NIGHT"))


Databricks data profile. Run in Databricks to view.